In [175]:
from monte_carlo import MonteCarloPricing
from black_scholes import OptionPrice
from unittest.mock import patch
import numpy as np
import matplotlib.pyplot as plt


# Initial Monte Carlo Pricing
Pricing of European and American Puts and Calls

In [205]:
from monte_carlo import *
mc = MonteCarloPricing(S_0=100, X=100, r=0.05, sigma=0.2, T=1.0, num_paths=1000, steps=252)

price_e, stderr_e = mc.european(call=True, antithetic=True)
print(f"Price {price_e:.4f} ± {1.96*stderr_e:.4f} (95% CI)")

price_a, stderr_a = mc.american(call=True, antithetic=True)
print(f"Price {price_a:.4f} ± {1.96*stderr_a:.4f} (95% CI)")

price_e, stderr_e = mc.european(call=False, antithetic=True)
print(f"Price {price_e:.4f} ± {1.96*stderr_e:.4f} (95% CI)")

price_a, stderr_a = mc.american(call=False, antithetic=True)
print(f"Price {price_a:.4f} ± {1.96*stderr_a:.4f} (95% CI)")

Price 10.5619 ± 0.9400 (95% CI)
Price 10.6192 ± 0.8623 (95% CI)
Price 5.5850 ± 0.5368 (95% CI)
Price 6.1637 ± 0.4754 (95% CI)


# Finite Difference Approximation of Sensitivities
Finding the sensitivities using Central Difference Estimate of European Call Option (European for now as American options require further computation)

In [195]:
from mc_finite_difference import MonteCarloFiniteDifference

mc = MonteCarloPricing(S_0=100, X=100, r=0.05, sigma=0.2, T=1.0, num_paths=10000, steps=252)
mc_fd = MonteCarloFiniteDifference(pricer=mc, call=True, antithetic=True, S0_bump=1e-2, sigma_bump=1e-2, r_bump=1e-3)

price, stderr = mc_fd.price()
print(f"Price {price:.4f} ± {1.96*stderr:.4f} (95% CI)")

delta = mc_fd.delta()
vega = mc_fd.vega()
rho = mc_fd.rho()
print(f"Delta: {delta:.4f}")
print(f"Vega: {vega:.4f}")
print(f"Rho: {rho:.4f}")

Price 10.4020 ± 0.2866 (95% CI)
Delta: 0.6363
Vega: 37.2055
Rho: 53.2390
